In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('datos/datos_filtrados_sin_encoding.csv')

In [10]:
display(df[df['is_training']==True]['genus'].value_counts())
len(df[df['is_training']==True])

genus
Flavobacterium       120
Bifidobacterium      120
Rhizobium            120
Microbacterium       120
Mesorhizobium        120
Acinetobacter        120
Collinsella          120
Pelagibacter_A       120
Arthrobacter         120
Nocardia             120
Micromonospora       120
Vibrio               120
Chryseobacterium     120
Mycobacterium        120
Nocardioides         120
Corynebacterium      120
Pseudomonas_E        120
Pelagibacter         120
Bradyrhizobium       120
Sphingomonas         120
Streptomyces         120
Prochlorococcus_A    120
Streptococcus        120
Paraburkholderia     118
Polynucleobacter     113
Pedobacter           112
Prevotella           105
Paracoccus           103
Novosphingobium      102
Methylobacterium     101
Name: count, dtype: int64

3514

In [ ]:
display(df[df['is_training']==False]['genus'].value_counts())
len(df[df['is_training']==False])

genus
Streptomyces         802
Pelagibacter         657
Pseudomonas_E        560
Streptococcus        343
Mycobacterium        257
Flavobacterium       245
Microbacterium       162
Prochlorococcus_A    113
Bradyrhizobium       109
Sphingomonas         104
Corynebacterium       81
Vibrio                63
Arthrobacter          60
Chryseobacterium      52
Acinetobacter         50
Nocardioides          49
Rhizobium             29
Collinsella           28
Micromonospora        23
Mesorhizobium         22
Nocardia              17
Bifidobacterium        8
Pelagibacter_A         4
Name: count, dtype: int64

3838

In [ ]:
display(df['genus'].value_counts())
len(df)

genus
Streptomyces         922
Pelagibacter         777
Pseudomonas_E        680
Streptococcus        463
Mycobacterium        377
Flavobacterium       365
Microbacterium       282
Prochlorococcus_A    233
Bradyrhizobium       229
Sphingomonas         224
Corynebacterium      201
Vibrio               183
Arthrobacter         180
Chryseobacterium     172
Acinetobacter        170
Nocardioides         169
Rhizobium            149
Collinsella          148
Micromonospora       143
Mesorhizobium        142
Nocardia             137
Bifidobacterium      128
Pelagibacter_A       124
Paraburkholderia     118
Polynucleobacter     113
Pedobacter           112
Prevotella           105
Paracoccus           103
Novosphingobium      102
Methylobacterium     101
Name: count, dtype: int64

7352

: 